In [1]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import numpy as np
import math
import collections
import jieba
import re
import os
from tensorflow.contrib.tensorboard.plugins import projector

In [2]:
# step 1 讀取停用詞
stop_words = []
f = open('stop_words.txt' , encoding = 'utf-8')
lines = f.readlines()
count = 0
for line in lines:
    line = line.replace('\n' , '')
    stop_words.append(line)

In [3]:
# step2 讀取文本，預處理，分詞，得到詞典
raw_word_list = []
sentence_list = []
f = open('novel_large.txt' , encoding = 'utf-8')
lines = f.readlines()
for i , sentence in enumerate(lines):
    sentence = sentence.replace('\n' , '')
    sentence = sentence.replace(' ' , '')
    if len(sentence) > 0: # 如果句子非空，才處理
        sentence_cut = list(jieba.cut(sentence , cut_all = False))
        sentence_temp = []
        for j , word in enumerate(sentence_cut):
            if word not in stop_words and word not in ['qingkan520' , 'www' , 'com' , 'http' ,  '本章'  , '免费']:
                if len(re.findall('第[\u4e00-\u9fa5]+章' , word)) == 0 : # 第一章、第二章、第二章...諸如此類的都不看
                    raw_word_list.append(word)
                    sentence_temp.append(word)
        if len(sentence_temp) != 0:         
            sentence_list.append(sentence_temp) 

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\peng\AppData\Local\Temp\jieba.cache
Loading model cost 0.817 seconds.
Prefix dict has been built succesfully.


In [4]:
# Step 3: Build the dictionary and replace rare words with UNK token.
# 為求簡單，先以raw_word_list來當作為當作訓練資料。但用sentence_list來當作訓練資料應該會比較好

# 計算詞頻
count = {}
for word in raw_word_list:
    if word in count.keys():
        count[word] += 1
    elif word not in count.keys():
        count[word] = 1
count = sorted(count.items() , key = lambda x : x[1] , reverse = True) # 由詞出現的次數由高排到低
count = [('UNK' , -1)] + count
vocabulary_size = 30000
count = count[:vocabulary_size] # 選出前30000個最常出現的詞

# 建立詞庫
vocab_to_int = {}
for i , word in enumerate(count):
    vocab_to_int[word[0]] = i
int_to_vocab = dict(zip(vocab_to_int.values() , vocab_to_int.keys()))

# 每raw_word_list的每個詞映射為一個數字ID
data = []
unk_count = 0
for word in raw_word_list:
    if word in vocab_to_int.keys():
        data.append(vocab_to_int[word])
    elif word not in vocab_to_int.keys():
        unk_count += 1 # 計算raw_word_list中不在vocab_to_int的詞的樹木
        data.append(vocab_to_int['UNK']) 

count = [list(content) for content in count] # 改成list格式，方便改count裡面的值
count[0][1] = unk_count
print('前5個最常出現的詞 : {}'.format(count[:5]))

前5個最常出現的詞 : [['UNK', 2705], ['萧炎', 16390], ['药', 6508], ['道', 5796], ['色', 4946]]


In [5]:
"""
選擇文本其中的 3 個詞，並選擇這3個詞的上文2個詞、下文2個詞

第 1 次訓練->
index : 0 , 1 , 2 
input_data:  data[0] , data[0] , data[1] , data[1] , data[1] , data[2] , data[2] , data[2] , data[2] 
output_data: data[1] , data[2] , data[0] , data[2] , data[3] , data[0] , data[1] , data[3] , data[4] 

第 2 次訓練->
index : 3 , 4 , 5
input_data:  data[3] , data[3] , data[3] , data[3] , data[4] , data[4] , data[4] , data[4] , data[5] , data[5] , data[5] , data[5] 
output_data: data[1] , data[2] , data[4] , data[5] , data[2] , data[3] , data[5] , data[6] , data[3] , data[4] , data[6] , data[7] 

第 3 次訓練->
index : 6 , 7 , 8
input_data:  data[6] , data[6] , data[6] , data[6] , data[7] , data[7] , data[7] , data[7] , data[8] , data[8] , data[8] , data[8] 
output_data: data[4] , data[5] , data[7] , data[8] , data[5] , data[6] , data[8] , data[9] , data[6] , data[7] , data[9] , data[10] 

.
.
.

第 334 次訓練->
index : 999 , 0 , 1
input_data:  data[999] , data[999] , data[0] , data[0] , data[1] , data[1] , data[1] 
output_data: data[997] , data[998] , data[1] , data[2] , data[0] , data[2] , data[3] 
"""

# Step 4: Function to generate a training batch for the skip-gram model.
# context_num_word : 決定上文與下文要選幾個字
global index
index = collections.deque(np.arange(0 , len(data)))
data_ = np.array(data)
def generate_batch(batch_size , context_num_word):
    global index
    input_data = []    
    label = []
    for j in range(0 , batch_size): # 一個batch抓32個詞，而一個詞最多會有2個上文、2個下文
        label_temp = np.arange(index[j] - context_num_word , index[j] + context_num_word + 1)
        label_temp = label_temp[label_temp >= 0]         # ex:在第1個詞前面不會有上文
        label_temp = label_temp[label_temp < len(index)] # ex:在第最後一個詞後面不會有下文
        label_temp = list(label_temp)
        label_temp.remove(index[j])
        label.extend(list(data_[np.array(label_temp)]))
        
        input_data_temp = []
        for _ in range(0 , len(label_temp)):
            input_data_temp.append(data_[index[j]])
        input_data.extend(input_data_temp)   
                    
    index.rotate(-batch_size) 
    
    return np.array(input_data) , np.expand_dims(np.array(label) , axis  = 1)

In [6]:
# Step 5: Build and train a skip-gram model.
batch_size = 130
embedding_size = 128  # Dimension of the embedding vector.
context_num_word = 2  # How many words to consider left and right.
num_sampled = 200     # Number of negative examples to sample.
vocabulary_size = len(vocab_to_int)

valid_size = 16  # 查看一下與某個詞相似的前16個詞
valid_window = 100 
np.random.seed(0)
valid_examples = np.random.choice(valid_window , valid_size , replace = False)


# Input data.
with tf.name_scope('inputs'):
    train_inputs = tf.placeholder(tf.int32 , shape = [None , ])
    train_labels = tf.placeholder(tf.int32 , shape = [None , 1])
    valid_dataset = tf.constant(valid_examples , dtype = tf.int32)


# Look up embeddings for inputs.
with tf.name_scope('embeddings'):
    embeddings = tf.Variable(tf.random_uniform([vocabulary_size , embedding_size] , -1.0 , 1.0))
    # 例如train_inputs第1個數字為'10'，代表是所有50000個詞中第10個詞，所以要從這50000個詞中，找出這第10個詞
    # 也就是50000組128維的向量(embeddings)中，把第10個字的128維的向量找出來
    embed = tf.nn.embedding_lookup(embeddings , train_inputs) 

# Construct the variables for the NCE loss
with tf.name_scope('weights'):
    nce_weights = tf.Variable(tf.truncated_normal([vocabulary_size , embedding_size],
                              stddev = 1.0 / math.sqrt(embedding_size)))
with tf.name_scope('biases'):
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

# Compute the average NCE loss for the batch.
# tf.nce_loss automatically draws a new sample of the negative labels each
# time we evaluate the loss.
# Explanation of the meaning of NCE loss : http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/
# 若label的詞是"the"，那就把"the"當作正樣本(1)，另外再從所有詞中選num_sampled個詞當作負樣本(0)，這樣可以讓訓練的結果更快
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.nce_loss(weights = nce_weights ,
                                         biases = nce_biases ,
                                         labels = train_labels ,
                                         inputs = embed ,
                                         num_sampled = num_sampled ,
                                         num_classes = vocabulary_size))

# Add the loss value as a scalar to summary.
tf.summary.scalar('loss', loss)

# Construct the SGD optimizer using a learning rate of 1.0.
with tf.name_scope('optimizer'):
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

<img src="相似度.JPG" style="width:550px;height:280px;float:middle">

In [7]:
# Compute the cosine similarity between minibatch examples and all embeddings.
norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings) , axis = 1 , keep_dims = True)) # [vocabulary_size , 1]
normalized_embeddings = embeddings / norm
valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings , valid_dataset)

# normalized_embeddings裏頭每個詞嵌向量與valid_embeddings每個詞嵌向量做內積
# 但其實normalized_embeddings中的每一個向量都已經是單位向量，因此內積計算的結果其實就是兩個詞嵌向量的cos(夾角)
# similarity(m , n)，代表valid_embeddings中第m個詞與normalized_embeddings第n個詞的相似度
similarity = tf.matmul(valid_embeddings , normalized_embeddings , transpose_b = True)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [8]:
# Step 6: Begin training.
sess = tf.Session() 
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('logs/' , sess.graph)
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

num_steps = 50000
average_loss = 0
for step in range(0 , num_steps):
    batch_inputs, batch_labels = generate_batch(batch_size , context_num_word)
    feed_dict = {train_inputs: batch_inputs , train_labels : batch_labels}

    _ , summary, loss_val = sess.run([optimizer , merged , loss] , feed_dict = feed_dict)
    
    average_loss += loss_val

    # Add returned summaries to writer in each step.
    writer.add_summary(summary, step)

    if step % 2000 == 0:
        if step > 0:
            average_loss = average_loss / 2000
        # The average loss is an estimate of the loss over the last 2000 batches.
        print('\nstep : {}'.format(step))
        print('Average loss : {} '.format(average_loss))
        average_loss = 0

    # Note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 2000 == 0:
        similar = sess.run(similarity)       
        for i in range(0 , valid_size):
            valid_word = int_to_vocab[valid_examples[i]]
            top_k = 8  # number of nearest neighbors
            nearest = (-similar[i , :]).argsort()[0 : top_k + 1]
            log_str = 'Nearest to {} : '.format(valid_word)
            
            close_word = []
            for k in range(0 , top_k):
                close_word.append(int_to_vocab[nearest[k]])
            log_str += ' , '.join(close_word)   
            
            print('=' * 90)    
            print(log_str)

# Write corresponding labels for the embeddings.
f = open('logs' + '/metadata.tsv' , 'w' , encoding = 'utf-8')
for i in range(0 , vocabulary_size):
    f.write(int_to_vocab[i] + '\n')

# Save the model for checkpoints.
saver.save(sess , './logs/' , global_step = step)

# Create a configuration for visualizing embeddings with the labels in TensorBoard.
config = projector.ProjectorConfig()
embedding_conf = config.embeddings.add()
embedding_conf.tensor_name = embeddings.name
embedding_conf.metadata_path = os.getcwd() + '//logs//metadata.tsv'
projector.visualize_embeddings(writer, config)

writer.close()


step : 0
Average loss : 689.859375 
Nearest to 炼 : 炼 , 砸过来 , 没事 , 强点 , 拖泥带水 , 东和米 , 轰掉 , 日子
Nearest to 帝国 : 帝国 , 自得 , 十多道 , 失守 , 万族 , 二十四 , 无存 , 娇声
Nearest to 药 : 药 , 位及 , 仰头望 , 余晖 , 叫雪妮 , 中翻 , 捷足先登 , 该狠
Nearest to 一道 : 一道 , 竟是 , 伤疤 , 之翼 , 催动 , 章炼 , 东三人 , 大张
Nearest to 云岚宗 : 云岚宗 , 大手笔 , 思念 , 用小玉棍 , 退势 , 情有可原 , 那任 , 前三天
Nearest to 勐然 : 勐然 , 小拳头 , 草能 , 搭上去 , 单挑 , 出面 , 渔人 , 转眼
Nearest to 笑 : 笑 , 显威 , 风啸 , 十头 , 寒铁 , 蓝 , 使点 , 手来
Nearest to 无奈 : 无奈 , 云草 , 之危 , 低身 , 紫袖 , 充耳不闻 , 地红 , 掩着
Nearest to 面前 : 面前 , 面目全非 , 曲拢 , 携带 , 二十一岁 , 小弟 , 游向 , 照样
Nearest to 院 : 院 , 冰枪 , 设法 , 裤子 , 暂停 , 苦中作乐 , 宽敞 , 萧家血
Nearest to 摇 : 摇 , 萧炎弟 , 通缉 , 功亏一篑 , 涂抹 , 修补 , 单一 , 议事厅
Nearest to 勐 : 勐 , 四五年 , 从椅 , 过生日 , 注定 , 满头 , 赶过去 , 蒙喇
Nearest to 吟 : 吟 , 被帝 , 葛 , 开大 , 湿润 , 这炼 , 一瞧 , 小房间
Nearest to 斗气 : 斗气 , 燥热 , 沙浪 , 疑似 , 之处 , 那烧 , 鼎力相助 , 地轻
Nearest to 目光 : 目光 , 欠上 , 浓得 , 逃走 , 老阴声 , 黑石 , 谷主 , 越牢
Nearest to 脸 : 脸 , 中之时 , 棱白 , 山峭 , 两点 , 血战 , 将漠 , 光明正大

step : 2000
Average loss : 122.93353064250947 
Nearest to 炼 : 炼 , 岩浆 , 水


step : 8000
Average loss : 6.673225182056427 
Nearest to 炼 : 炼 , 没事 , 光团 , 对话 , 黑球 , 蝎 , 嚎叫 , 银裙
Nearest to 帝国 : 帝国 , 鹰翼 , 净莲 , 光团 , 盾牌 , 自得 , 二十四 , 蝎
Nearest to 药 : 药 , 鹰翼 , 鼠 , 护喉 , 黑帘 , 暗流 , 成三纹 , 火旋
Nearest to 一道 : 一道 , 之翼 , 竟是 , 石门 , 系 , 催动 , 龙气 , 跺
Nearest to 云岚宗 : 云岚宗 , 蝎 , 玉片 , 大手笔 , 锥 , 泥沙 , 这名 , 水镜
Nearest to 勐然 : 勐然 , 堡垒 , 郝 , 小拳头 , 草能 , 单挑 , 小小 , 管家
Nearest to 笑 : 笑 , 显威 , 十头 , 说起 , 水镜 , 蓝 , 火旋 , 风啸
Nearest to 无奈 : 无奈 , 云草 , 之危 , 龙气 , 低身 , 涨 , 触须 , 双翼
Nearest to 面前 : 面前 , 海胆 , 难吃 , 凌影点 , 携带 , 晶片 , 火旋 , 泥沙
Nearest to 院 : 院 , 冰枪 , 丹香 , 设法 , 宽敞 , 遗物 , 各方 , 光团
Nearest to 摇 : 摇 , 修补 , 萧炎弟 , 获 , 难吃 , 园陵 , 功亏一篑 , 泥沙
Nearest to 勐 : 勐 , 冰刃 , 火旋 , 玉尺 , 摊位 , 获 , 帝品 , 鼠
Nearest to 吟 : 吟 , 冰柱 , 湿润 , 小房间 , 这炼 , 人影 , 开大 , 叶家
Nearest to 斗气 : 斗气 , 沙浪 , 燥热 , 材都 , 介绍 , 迈步 , 岩浆 , 五十多天
Nearest to 目光 : 目光 , 帝品 , 凝血 , 第一百七 , 躲起来 , 逃走 , 银盾 , 光团
Nearest to 脸 : 脸 , 无效 , 血战 , 植物 , 将漠 , 膀子 , 棱白 , 那缕

step : 10000
Average loss : 6.165368473529815 
Nearest to 炼 : 炼 , 没事 , 光团 , 对话 , 蝎 , 黑球 , 嚎叫 , 银裙
Nea


step : 16000
Average loss : 5.894640370607376 
Nearest to 炼 : 炼 , 对话 , 没事 , 光团 , 嚎叫 , 蝎 , 黑球 , 大爷
Nearest to 帝国 : 帝国 , 鹰翼 , 净莲 , 自得 , 光团 , 盾牌 , 娇声 , 差点
Nearest to 药 : 药 , 鹰翼 , 护喉 , 成三纹 , 鼠 , 黑帘 , 火旋 , 暗流
Nearest to 一道 : 一道 , 之翼 , 跺 , 催动 , 竟是 , 支 , 石门 , 系
Nearest to 云岚宗 : 云岚宗 , 蝎 , 锥 , 泥沙 , 大手笔 , 真不知道 , 这名 , 赛中
Nearest to 勐然 : 勐然 , 堡垒 , 小拳头 , 草能 , 单挑 , 郝 , 搭上去 , 弓
Nearest to 笑 : 笑 , 显威 , 十头 , 说起 , 风啸 , 花容 , 无奈 , 咬着牙
Nearest to 无奈 : 无奈 , 云草 , 笑 , 之危 , 低身 , 涨 , 己身 , 不早了
Nearest to 面前 : 面前 , 凌影点 , 海胆 , 难吃 , 携带 , 泥沙 , 晶片 , 火旋
Nearest to 院 : 院 , 冰枪 , 丹香 , 设法 , 宽敞 , 各方 , 遗物 , 萧家血
Nearest to 摇 : 摇 , 萧炎弟 , 修补 , 获 , 园陵 , 功亏一篑 , 难吃 , 四岁
Nearest to 勐 : 勐 , 冰刃 , 火旋 , 摊位 , 玉尺 , 栽 , 甩头 , 蒙喇
Nearest to 吟 : 吟 , 湿润 , 开大 , 冰柱 , 神秘莫测 , 小房间 , 弯下身 , 人影
Nearest to 斗气 : 斗气 , 沙浪 , 材都 , 燥热 , 迈步 , 介绍 , 五十多天 , 聚焦
Nearest to 目光 : 目光 , 帝品 , 凝血 , 视线 , 银盾 , 稍作 , 逃走 , 躲起来
Nearest to 脸 : 脸 , 无效 , 将漠 , 膀子 , 血战 , 植物 , 不及 , 撤退

step : 18000
Average loss : 5.84557857632637 
Nearest to 炼 : 炼 , 对话 , 没事 , 光团 , 嚎叫 , 蝎 , 黑球 , 


step : 24000
Average loss : 5.726925113201141 
Nearest to 炼 : 炼 , 对话 , 嚎叫 , 没事 , 光团 , 蝎 , 大爷 , 黑球
Nearest to 帝国 : 帝国 , 鹰翼 , 自得 , 想 , 净莲 , 盾牌 , 光团 , 差点
Nearest to 药 : 药 , 鹰翼 , 护喉 , 火旋 , 黑帘 , 成三纹 , 一握 , 股温
Nearest to 一道 : 一道 , 之翼 , 支 , 跺 , 催动 , 倒吸 , 竟是 , 毒矛
Nearest to 云岚宗 : 云岚宗 , 真不知道 , 蝎 , 中探 , 一入 , 用小玉棍 , 这名 , 赛中
Nearest to 勐然 : 勐然 , 堡垒 , 小拳头 , 草能 , 单挑 , 脱困 , 搭上去 , 弓
Nearest to 笑 : 笑 , 显威 , 十头 , 花容 , 风啸 , 说起 , 无奈 , 咬着牙
Nearest to 无奈 : 无奈 , 云草 , 笑 , 之危 , 低身 , 己身 , 涨 , 不早了
Nearest to 面前 : 面前 , 凌影点 , 难吃 , 海胆 , 携带 , 泥沙 , 火旋 , 晶片
Nearest to 院 : 院 , 萧家血 , 冰枪 , 各方 , 宽敞 , 设法 , 丹香 , 裤子
Nearest to 摇 : 摇 , 萧炎弟 , 修补 , 园陵 , 获 , 功亏一篑 , 激起 , 四岁
Nearest to 勐 : 勐 , 冰刃 , 火旋 , 摊位 , 玉尺 , 栽 , 蒙喇 , 甩头
Nearest to 吟 : 吟 , 神秘莫测 , 开大 , 湿润 , 冰柱 , 小房间 , 弯下身 , 招出
Nearest to 斗气 : 斗气 , 沙浪 , 材都 , 燥热 , 聚焦 , 迈步 , 介绍 , 五十多天
Nearest to 目光 : 目光 , 帝品 , 视线 , 凝血 , 银盾 , 稍作 , 逃走 , 计划
Nearest to 脸 : 脸 , 无效 , 将漠 , 膀子 , 血战 , 苍白 , 不及 , 五十九

step : 26000
Average loss : 5.706859858751297 
Nearest to 炼 : 炼 , 对话 , 嚎叫 , 没事 , 蝎 , 大爷 , 


step : 32000
Average loss : 5.611574798345566 
Nearest to 炼 : 炼 , 对话 , 嚎叫 , 没事 , 大爷 , 蝎 , 光团 , 城是
Nearest to 帝国 : 帝国 , 自得 , 想 , 鹰翼 , 加玛 , 盾牌 , 想当初 , 差点
Nearest to 药 : 药 , 鹰翼 , 股温 , 火旋 , 护喉 , 黑帘 , 一握 , 炼制
Nearest to 一道 : 一道 , 之翼 , 支 , 倒吸 , 跺 , 催动 , 毒矛 , 讨人
Nearest to 云岚宗 : 云岚宗 , 真不知道 , 家族 , 一入 , 中探 , 这名 , 用小玉棍 , 一无所获
Nearest to 勐然 : 勐然 , 草能 , 小拳头 , 堡垒 , 脱困 , 单挑 , 过往 , 搭上去
Nearest to 笑 : 笑 , 显威 , 花容 , 风啸 , 十头 , 说起 , 咬着牙 , 无奈
Nearest to 无奈 : 无奈 , 云草 , 笑 , 之危 , 低身 , 叹息 , 只得 , 己身
Nearest to 面前 : 面前 , 凌影点 , 携带 , 难吃 , 几段 , 海胆 , 泥沙 , 图腾
Nearest to 院 : 院 , 萧家血 , 内院 , 裤子 , 各方 , 设法 , 宽敞 , 拼了命
Nearest to 摇 : 摇 , 萧炎弟 , 修补 , 激起 , 园陵 , 获 , 功亏一篑 , 四岁
Nearest to 勐 : 勐 , 冰刃 , 摊位 , 火旋 , 玉尺 , 蒙喇 , 甩头 , 栽
Nearest to 吟 : 吟 , 神秘莫测 , 开大 , 湿润 , 小房间 , 冰柱 , 招出 , 弯下身
Nearest to 斗气 : 斗气 , 沙浪 , 材都 , 燥热 , 聚焦 , 迈步 , 五十多天 , 介绍
Nearest to 目光 : 目光 , 视线 , 帝品 , 稍作 , 银盾 , 凝血 , 逃走 , 细语
Nearest to 脸 : 脸 , 无效 , 将漠 , 膀子 , 苍白 , 血战 , 不及 , 脸庞

step : 34000
Average loss : 5.6180727005004885 
Nearest to 炼 : 炼 , 对话 , 嚎叫 , 没事 , 大爷 ,


step : 40000
Average loss : 5.528401792287826 
Nearest to 炼 : 炼 , 对话 , 嚎叫 , 大爷 , 疗伤 , 没事 , 蝎 , 城是
Nearest to 帝国 : 帝国 , 加玛 , 自得 , 想 , 鹰翼 , 盾牌 , 想当初 , 地疑
Nearest to 药 : 药 , 鹰翼 , 股温 , 炼制 , 火旋 , 一握 , 护喉 , 成三纹
Nearest to 一道 : 一道 , 支 , 之翼 , 倒吸 , 毒矛 , 以异火 , 催动 , 跺
Nearest to 云岚宗 : 云岚宗 , 真不知道 , 家族 , 一入 , 这名 , 中探 , 一无所获 , 退势
Nearest to 勐然 : 勐然 , 草能 , 小拳头 , 堡垒 , 脱困 , 单挑 , 过往 , 再度
Nearest to 笑 : 笑 , 显威 , 花容 , 微笑 , 风啸 , 说起 , 咬着牙 , 十头
Nearest to 无奈 : 无奈 , 云草 , 只得 , 叹息 , 笑 , 之危 , 低身 , 淡笑
Nearest to 面前 : 面前 , 凌影点 , 携带 , 几段 , 难吃 , 图腾 , 上炼 , 泥沙
Nearest to 院 : 院 , 内院 , 萧家血 , 裤子 , 宽敞 , 各方 , 设法 , 差别
Nearest to 摇 : 摇 , 萧炎弟 , 修补 , 摇头 , 合四人 , 激起 , 四岁 , 获
Nearest to 勐 : 勐 , 冰刃 , 摊位 , 火旋 , 甩头 , 蒙喇 , 玉尺 , 帝品
Nearest to 吟 : 吟 , 神秘莫测 , 开大 , 小房间 , 招出 , 睁眼 , 湿润 , 冰柱
Nearest to 斗气 : 斗气 , 沙浪 , 材都 , 燥热 , 聚焦 , 盛有 , 再开 , 迈步
Nearest to 目光 : 目光 , 视线 , 帝品 , 稍作 , 银盾 , 细语 , 逃走 , 欠上
Nearest to 脸 : 脸 , 无效 , 将漠 , 膀子 , 苍白 , 脸庞 , 不及 , 血战

step : 42000
Average loss : 5.538026232481003 
Nearest to 炼 : 炼 , 对话 , 嚎叫 , 疗伤 , 大爷 , 没事 , 


step : 48000
Average loss : 5.467574532270431 
Nearest to 炼 : 炼 , 对话 , 嚎叫 , 疗伤 , 师 , 大爷 , 没事 , 城是
Nearest to 帝国 : 帝国 , 加玛 , 自得 , 想 , 想当初 , 盾牌 , 鹰翼 , 地疑
Nearest to 药 : 药 , 股温 , 鹰翼 , 火旋 , 炼制 , 一握 , 成三纹 , 鼠
Nearest to 一道 : 一道 , 支 , 之翼 , 倒吸 , 以异火 , 毒矛 , 催动 , 竟是
Nearest to 云岚宗 : 云岚宗 , 家族 , 真不知道 , 一入 , 中探 , 这名 , 一无所获 , 社会
Nearest to 勐然 : 勐然 , 草能 , 小拳头 , 脱困 , 堡垒 , 过往 , 单挑 , 骤然
Nearest to 笑 : 笑 , 显威 , 微笑 , 花容 , 风啸 , 咬着牙 , 说起 , 寒铁
Nearest to 无奈 : 无奈 , 只得 , 叹息 , 云草 , 淡笑 , 忍耐住 , 命来 , 之危
Nearest to 面前 : 面前 , 凌影点 , 图腾 , 几段 , 上炼 , 携带 , 帝品 , 激战
Nearest to 院 : 院 , 内院 , 萧家血 , 裤子 , 宽敞 , 各方 , 设法 , 差别
Nearest to 摇 : 摇 , 萧炎弟 , 摇头 , 修补 , 合四人 , 四岁 , 激起 , 获
Nearest to 勐 : 勐 , 冰刃 , 摊位 , 火旋 , 甩头 , 帝品 , 蒙喇 , 玉尺
Nearest to 吟 : 吟 , 神秘莫测 , 小房间 , 开大 , 招出 , 睁眼 , 被帝 , 湿润
Nearest to 斗气 : 斗气 , 沙浪 , 材都 , 聚焦 , 燥热 , 再开 , 盛有 , 衣袍
Nearest to 目光 : 目光 , 视线 , 帝品 , 稍作 , 银盾 , 细语 , 逃走 , 讥笑
Nearest to 脸 : 脸 , 无效 , 将漠 , 膀子 , 苍白 , 脸庞 , 不及 , 血战
